# Setup

In [1]:
import loompy as lp
import anndata as ad
import fast_matrix_market as fmm
import pandas as pd
import scvelo as scv
import dynamo as dyn
import matplotlib.pyplot as plt
import os

# Create H5AD file

We provide merged dataset in `velocity.loom`.

In [4]:
adata = ad.read_loom("velocity.loom")

In [5]:
adata

AnnData object with n_obs × n_vars = 140260 × 36601
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'matrix', 'ambiguous', 'spliced', 'unspliced'

Make unique gene names and update cell names to match those from R

In [6]:
adata.obs.index

Index(['C744:AAAGGGCGTGTGGACAx', 'C744:AAACCCAGTGCCGAAAx',
       'C744:AAAGGGCGTTGGCCTGx', 'C744:AAACGCTTCAGATTGCx',
       'C744:AAACGCTGTGACACAGx', 'C744:AAACGCTAGAGGTCGTx',
       'C744:AAACGCTTCCACAGCGx', 'C744:AAACCCAAGAATTTGGx',
       'C744:AAACCCAGTAATTAGGx', 'C744:AAACGAAGTGGCTGAAx',
       ...
       'C751:TTTGTTGCAACACAGGx', 'C751:TTTGTTGAGTTTCGACx',
       'C751:TTTGGAGTCTGATTCTx', 'C751:TTTGTTGGTGGCTTGCx',
       'C751:TTTGGAGAGTGGTTAAx', 'C751:TTTGTTGGTCAATGGGx',
       'C751:TTTGTTGTCATCGCCTx', 'C751:TTTGGAGCAACACAGGx',
       'C751:TTTGTTGCAACTCGTAx', 'C751:TTTGTTGTCCACTAGAx'],
      dtype='object', length=140260)

In [7]:
new_index = []
for string in adata.obs.index:
    new_index.append(string.replace(":", "!!").replace("x", "-1"))

adata.obs.index = new_index

In [8]:
adata.var_names_make_unique()

In [9]:
adata.obs.index.name = "cells"
adata.var.index.name = "genes"

## Save file

In [10]:
adata.write_h5ad("velocity.h5ad")

In [11]:
adata

AnnData object with n_obs × n_vars = 140260 × 36601
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'matrix', 'ambiguous', 'spliced', 'unspliced'

# Create subsets

In [2]:
adata = ad.read_h5ad("velocity.h5ad")

## Vascular

These data are exports from R, see `Objects_preparations.Rmd`

In [3]:
adata_subset = ad.AnnData(X = fmm.mmread("vascular.mtx"))
cellids = pd.read_csv("vascular.cells", header = None)
genes = pd.read_csv("vascular.genes", header = None)

In [6]:
anno = pd.read_csv("vascular.annotation", header = None)
anno.index = anno[1]
anno.index.name = "cells"
anno

,0,1
cells,,
Donor_02!!9984!!TCATTACAGTTAGAAC-1,arEC,Donor_02!!9984!!TCATTACAGTTAGAAC-1
Donor_05!!9972!!TCTTCCTAGCTCCCTT-1,arEC,Donor_05!!9972!!TCTTCCTAGCTCCCTT-1
Donor_02!!9984!!CCTCAGTCATGATAGA-1,arEC,Donor_02!!9984!!CCTCAGTCATGATAGA-1
Donor_02!!9985!!AAAGTGACACCTATCC-1,arEC,Donor_02!!9985!!AAAGTGACACCTATCC-1
Donor_02!!9985!!CATACAGCACGACGTC-1,arEC,Donor_02!!9985!!CATACAGCACGACGTC-1
...,...,...
Donor_88!!C750!!CATGAGTTCGCGTGCA-1,Pericytes,Donor_88!!C750!!CATGAGTTCGCGTGCA-1
Donor_88!!C750!!GGGTCACCACTGTCGG-1,Pericytes,Donor_88!!C750!!GGGTCACCACTGTCGG-1
Donor_88!!C751!!GCACATAGTGGTAATA-1,arEC,Donor_88!!C751!!GCACATAGTGGTAATA-1


In [7]:
adata_subset.obs.index = cellids[0]
adata_subset.var.index = genes[0]
adata_subset.obs["annotation"] = anno[1]
adata_subset.obs.index.name = "cells"
adata_subset.var.index.name = "genes"

In [9]:
emb = pd.read_csv("vascular.embedding", header = None)
emb.index = anno[1]
emb.index.name = "cells"
emb

,0,1
cells,,
Donor_02!!9984!!TCATTACAGTTAGAAC-1,14.043085,-50.135545
Donor_05!!9972!!TCTTCCTAGCTCCCTT-1,-73.528391,-6.459791
Donor_02!!9984!!CCTCAGTCATGATAGA-1,9.088842,-48.238909
Donor_02!!9985!!AAAGTGACACCTATCC-1,-38.991121,-18.001077
Donor_02!!9985!!CATACAGCACGACGTC-1,50.805575,11.390331
...,...,...
Donor_88!!C750!!CATGAGTTCGCGTGCA-1,-5.171450,48.436449
Donor_88!!C750!!GGGTCACCACTGTCGG-1,23.772841,-2.359125
Donor_88!!C751!!GCACATAGTGGTAATA-1,-33.482225,21.748196


In [10]:
adata_subset.obsm["X_umap"] = emb.to_numpy()

In [11]:
adata_subset.X = adata_subset.X.tocsr()

### Merge with loom

Adjust cells and genes

In [12]:
new_idx = []
for string in adata_subset.obs.index:
    new_idx.append(string.split(sep = "!!", maxsplit = 1)[1])

adata_subset.obs.index = new_idx

In [13]:
cell_list = list(set(adata.obs.index.to_list()) & set(adata_subset.obs.index.to_list()))
gene_list = list(set(adata.var.index.to_list()) & set(adata_subset.var.index.to_list()))
adata2 = adata[cell_list, gene_list]
adata_subset2 = adata_subset[cell_list, gene_list]

In [14]:
adata2

View of AnnData object with n_obs × n_vars = 26887 × 36389
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [15]:
adata_subset2

View of AnnData object with n_obs × n_vars = 26887 × 36389
    obs: 'annotation'
    obsm: 'X_umap'

In [16]:
adata_velo = scv.utils.merge(adata_subset2, adata2)

In [17]:
adata_velo

AnnData object with n_obs × n_vars = 26887 × 36389
    obs: 'annotation', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    obsm: 'X_umap'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [ ]:
adata_velo.write_h5ad("vascular.h5ad")

## Adipocytes, ASPCs, macrophages

These data are exports from R, see `Objects_preparations.Rmd`

In [3]:
adata_subset = ad.AnnData(X = fmm.mmread("aspc_adi_macro.mtx"))
cellids = pd.read_csv("aspc_adi_macro.cells", header = None)
genes = pd.read_csv("aspc_adi_macro.genes", header = None)

In [18]:
anno = pd.read_csv("aspc_adi_macro.annotation", header = None)
anno.index = anno[0]
anno.index.name = "cells"
anno

,0,1
cells,,
Donor_02!!9984!!AATTTCCTCGTTCTAT-1,Donor_02!!9984!!AATTTCCTCGTTCTAT-1,Adipocytes
Donor_02!!9984!!TCGACCTCAGTATTCG-1,Donor_02!!9984!!TCGACCTCAGTATTCG-1,Adipocytes
Donor_02!!9984!!TTATTGCGTTCTCCAC-1,Donor_02!!9984!!TTATTGCGTTCTCCAC-1,Adipocytes
Donor_02!!9984!!AACAACCCATTCTGTT-1,Donor_02!!9984!!AACAACCCATTCTGTT-1,LAM
Donor_02!!9984!!AGATAGAGTTGCATGT-1,Donor_02!!9984!!AGATAGAGTTGCATGT-1,Adipocytes
...,...,...
Donor_88!!C751!!CGCGTGAGTCTAGGCC-1,Donor_88!!C751!!CGCGTGAGTCTAGGCC-1,Adipocytes
Donor_88!!C751!!CATCCCACAAATGAAC-1,Donor_88!!C751!!CATCCCACAAATGAAC-1,ATM
Donor_88!!C751!!AACGGGACATTGCAAC-1,Donor_88!!C751!!AACGGGACATTGCAAC-1,ASPC_DPP4


In [5]:
adata_subset.obs.index = cellids[0]
adata_subset.var.index = genes[0]
adata_subset.obs["annotation"] = anno[1]
adata_subset.obs.index.name = "cells"
adata_subset.var.index.name = "genes"

In [6]:
emb = pd.read_csv("aspc_adi_macro.embedding", header = None)
emb.index = anno[0]
emb.index.name = "cells"
emb

,0,1
cells,,
Donor_02!!9984!!AATTTCCTCGTTCTAT-1,0.000003,-2.045266
Donor_02!!9984!!TCGACCTCAGTATTCG-1,-0.057343,-1.398973
Donor_02!!9984!!TTATTGCGTTCTCCAC-1,-0.695034,-0.945508
Donor_02!!9984!!AACAACCCATTCTGTT-1,1.227550,-7.358194
Donor_02!!9984!!AGATAGAGTTGCATGT-1,0.723089,-1.886719
...,...,...
Donor_88!!C751!!CGCGTGAGTCTAGGCC-1,-1.313268,-1.416494
Donor_88!!C751!!CATCCCACAAATGAAC-1,4.817921,-4.887340
Donor_88!!C751!!AACGGGACATTGCAAC-1,-2.392691,3.082013


In [7]:
adata_subset.obsm["X_umap"] = emb.to_numpy()

In [8]:
adata_subset.X = adata_subset.X.tocsr()

### Merge with loom

Adjust cells and genes

In [9]:
new_idx = []
for string in adata_subset.obs.index:
    new_idx.append(string.split(sep = "!!", maxsplit = 1)[1])

adata_subset.obs.index = new_idx

In [10]:
cell_list = list(set(adata.obs.index.to_list()) & set(adata_subset.obs.index.to_list()))
gene_list = list(set(adata.var.index.to_list()) & set(adata_subset.var.index.to_list()))
adata2 = adata[cell_list, gene_list]
adata_subset2 = adata_subset[cell_list, gene_list]

In [11]:
adata2

View of AnnData object with n_obs × n_vars = 85121 × 36389
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [12]:
adata_subset2

View of AnnData object with n_obs × n_vars = 85121 × 36389
    obs: 'annotation'
    obsm: 'X_umap'

In [13]:
adata_velo = scv.utils.merge(adata_subset2, adata2)

In [14]:
adata_velo

AnnData object with n_obs × n_vars = 85121 × 36389
    obs: 'annotation', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    obsm: 'X_umap'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [15]:
adata_velo.write_h5ad("aspc_adi_macro.h5ad")